In [1]:
import numpy as np
import math
import unittest

In [2]:
def is_u_triangular(a:np.ndarray):
    return np.allclose(a[:, :4], np.triu(a[:, :4]))
def is_singular(a:np.ndarray):
    return np.linalg.det(a)
def sign(x):
    if x > 0:
        return 1
    elif x < 0:
        return -1
    return 0

In [64]:
def reflection_method(a:np.array, b:np.array) -> np.array:
    b = b.copy()
    a = a.copy()
    n, m = a.shape
    for i in range(n-1):
        s = sign(-a[i,i])*np.linalg.norm(a[:, i])
        etta = 1 / math.sqrt(2 * s * (s - a[i,i]))
        w = a[:,i][..., np.newaxis].copy()
        for j in range(i):
            w[j, 0] = 0
        w[i, 0] -= s
        w *= etta
        u = np.eye(n) - 2 * np.matmul(w, w.T)
        a = np.matmul(u, a)
        b = np.matmul(u, b)
    return backward_gauss_method(np.concatenate((a, b), axis=1))

In [4]:
def backward_gauss_method(a:np.array) -> np.array:
    a = a.copy()
    n, m = a.shape
    x = np.zeros((n, 1))
    for k in range(n-1, -1, -1):
        x[k] = (a[k, -1] - np.dot(a[k, k:n], x[k:n])) / a[k, k]
    return x

In [5]:
def rotation_method(a:np.array) -> np.array:
    a = a.copy()
    n, m = a.shape
    for k in range(0, m - 1):
        for j in range(k + 1, n):
            temp = math.sqrt(a[k,k] ** 2 + a[j,k] ** 2)
            cos = a[k,k] / (temp)
            sin = a[j,k] / (temp)
            a_i = cos * a[k] + sin * a[j]
            a_j = -sin * a[k] + cos * a[j]
            a[k] = a_i
            a[j] = a_j
    return backward_gauss_method(a)

In [6]:
test = np.array([[0.0002, 3.4, -1.8, 2.1, 2.3561],
                 [3.7, -0.9, 0.7, 5.7, 7.556],
                 [-1.9, -27, 5.3, 2.9, -3.051],
                 [18, 4.1, 7.3, -0.1, 12.334]]
                 )
res = rotation_method(test)
print(res)

[[0.5 ]
 [0.25]
 [0.33]
 [1.  ]]


In [7]:
print()


In [8]:
a = test[:, :4]
b = test[:,-1]
print(np.allclose(np.linalg.solve(a, b), res[:,0]))



True


In [9]:
test = np.array([
    [0.521, -0.296, 0, 0.04, -0.21, 0.24886],
    [-0.296, 0.7, 0.24, -0.32, 0.06, -0.7012],
    [0, 0.24, -0.35, 0.07, 0.21, 0.1347],
    [0.04, -0.32, 0.7, 0.49, -0.03, 0.4659],
    [-0.21, 0.06, 0.21, -0.03, 0.63, -0.1749]
                   ])
w = np.random.sample((4,1))
w /= np.linalg.norm(w)
print("Is vector \"w\" normalized: " + str(np.allclose(np.linalg.norm(w), 1)))
H = np.eye(4) - 2 * np.matmul(w, w.T)
print("Is \"u\" symmetric: " + str(np.allclose(H, H.T)))
print("Is \"u\" orthogonalized: " + str(np.allclose(np.matmul(H, H.T), np.eye(4))))

#print(np.allclose(, np.ones((4, 4))))

Is vector "w" normalized: True
Is "u" symmetric: True
Is "u" orthogonalized: True


In [10]:
print(reflection_method(test))

[[ 0.2415404 ]
 [-0.24623598]
 [-0.256537  ]
 [ 1.1610081 ]
 [-0.04976496]]


In [11]:
print(test[:, -1:])
print(test[:,:-1])

[[ 0.24886]
 [-0.7012 ]
 [ 0.1347 ]
 [ 0.4659 ]
 [-0.1749 ]]
[[ 0.521 -0.296  0.     0.04  -0.21 ]
 [-0.296  0.7    0.24  -0.32   0.06 ]
 [ 0.     0.24  -0.35   0.07   0.21 ]
 [ 0.04  -0.32   0.7    0.49  -0.03 ]
 [-0.21   0.06   0.21  -0.03   0.63 ]]


[[0.76661972]
 [0.05720294]
 [0.28601469]
 [0.57202939]]


In [14]:
u = np.eye(4) - 2 * np.matmul(w, w.T)
print(u)

[[-0.1754116  -0.0877058  -0.43852901 -0.87705802]
 [-0.0877058   0.99345565 -0.03272176 -0.06544352]
 [-0.43852901 -0.03272176  0.83639119 -0.32721762]
 [-0.87705802 -0.06544352 -0.32721762  0.34556476]]


In [61]:
import unittest
class TestBackwardGaussMethod(unittest.TestCase):
    def pattern(self, a, x):
        b = np.matmul(a, x)
        a_extend = np.append(a, b, axis=1)
        calc_x = backward_gauss_method(a_extend)
        is_close = np.allclose(
            calc_x,
            x
        )
        self.assertTrue(is_close)
    def test_5_x_5(self):
        a = np.array([
            [1, 2, 3, 4, 5],
            [0, 7, 6, 1, 2],
            [0, 0, 3, 5, 8],
            [0, 0, 0, 9, 2],
            [0, 0, 0, 0, 1]
        ], dtype='float64')
        x = np.array([[-5, 6, 8, -2, -4]]).T
        self.pattern(a, x)
    def test_3x3(self):
        a = np.array([
            [7, -2, 3],
            [0, 9, 0],
            [0, 0, -2]
        ], dtype='float64')
        x = np.array([[-1, 2, -6]]).T
        self.pattern(a, x)
class TestRotationMethod(unittest.TestCase):
    def setUp(self):
        print(type(self).__name__)
    def pattern(self, a, x, method_name):
        b = np.matmul(a, x)
        a_extend = np.append(a, b, axis=1)
        calc_x = rotation_method(a_extend)
        is_close = np.allclose(
            calc_x,
            x
        )
        discrepancy = b - np.matmul(a, calc_x)
        print(str(method_name), '\n', discrepancy)
        self.assertTrue(is_close)
    def test_symmetric(self):
        a = np.array([
            [0.521,-0.296, 0, 0.04, -0.21],
            [-0.296, 0.7, 0.24, -0.32, 0.06],
            [0, 0.24, -0.35, 0.07, 0.21],
            [0.04, -0.32, 0.7, 0.49, -0.03],
            [-0.21, 0.06, 0.21, -0.03, 0.63]
        ], dtype='float64')
        x = np.array([[1, -3, 4, 8, -1]]).T
        self.pattern(a, x, self.test_symmetric)
    def test_square(self):
        a = np.array([
            [2, 3, -4, 1],
            [1, -2, -5, 1],
            [5, -3, 1, -4],
            [10, 2, -1, 2]
        ], dtype='float64')
        x = np.array([[1, 3, -4, 2]], dtype='float64').T
        self.pattern(a, x, self.test_square)
    def test_hilbert(self):
        n = 5
        a = np.array([[1/(i + j - 1) for j in range(1, n+1)] for i in range(1, n + 1)])
        x = np.array([[-5, 6, 8, -2, -4]]).T
        self.pattern(a, x, self.test_hilbert)
class TestReflectionMethod(unittest.TestCase):
    def setUp(self):
        print(type(self).__name__)
    def pattern(self, a, x, method_name):
        b = np.matmul(a, x)
        a_extend = np.append(a, b, axis=1)
        calc_x = reflection_method(a_extend)
        is_close = np.allclose(
            calc_x,
            x
        )
        discrepancy = b - np.matmul(a, calc_x)
        print(str(method_name), '\n', discrepancy)
        self.assertTrue(is_close)
    def test_symmetric(self):
        a = np.array([
            [0.521,-0.296, 0, 0.04, -0.21],
            [-0.296, 0.7, 0.24, -0.32, 0.06],
            [0, 0.24, -0.35, 0.07, 0.21],
            [0.04, -0.32, 0.7, 0.49, -0.03],
            [-0.21, 0.06, 0.21, -0.03, 0.63]
        ], dtype='float64')
        x = np.array([[1, -3, 4, 8, -1]]).T
        self.pattern(a, x, self.test_symmetric)
    def test_square(self):
        a = np.array([
            [2, 3, -4, 1],
            [1, -2, -5, 1],
            [5, -3, 1, -4],
            [10, 2, -1, 2]
        ], dtype='float64')
        x = np.array([[1, 3, -4, 2]], dtype='float64').T
        self.pattern(a, x, self.test_square)
    def test_hilbert(self):
        n = 5
        a = np.array([[1/(i + j - 1) for j in range(1, n+1)] for i in range(1, n + 1)])
        x = np.array([[-5, 6, 8, -2, -4]]).T
        self.pattern(a, x, self.test_hilbert)
unittest.main(argv=[''], verbosity=2, exit=False)

test_3x3 (__main__.TestBackwardGaussMethod) ... ok
test_5_x_5 (__main__.TestBackwardGaussMethod) ... ok
test_hilbert (__main__.TestReflectionMethod) ... FAIL
test_square (__main__.TestReflectionMethod) ... FAIL
test_symmetric (__main__.TestReflectionMethod) ... FAIL
test_hilbert (__main__.TestRotationMethod) ... ok
test_square (__main__.TestRotationMethod) ... ok
test_symmetric (__main__.TestRotationMethod) ... ok

FAIL: test_hilbert (__main__.TestReflectionMethod)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-61-be504b517716>", line 105, in test_hilbert
    self.pattern(a, x, self.test_hilbert)
  File "<ipython-input-61-be504b517716>", line 81, in pattern
    self.assertTrue(is_close)
AssertionError: False is not true

FAIL: test_square (__main__.TestReflectionMethod)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-61-be504

TestReflectionMethod
<bound method TestReflectionMethod.test_hilbert of <__main__.TestReflectionMethod testMethod=test_hilbert>> 
 [[-0.02421435]
 [ 0.09699451]
 [ 0.01384976]
 [-0.04753712]
 [-0.08507607]]
TestReflectionMethod
<bound method TestReflectionMethod.test_square of <__main__.TestReflectionMethod testMethod=test_square>> 
 [[-0.50129495]
 [ 0.01584289]
 [-0.3345553 ]
 [ 0.26595235]]
TestReflectionMethod
<bound method TestReflectionMethod.test_symmetric of <__main__.TestReflectionMethod testMethod=test_symmetric>> 
 [[-0.06063187]
 [-0.16106731]
 [-0.02576008]
 [-0.312363  ]
 [ 0.01710571]]
TestRotationMethod
<bound method TestRotationMethod.test_hilbert of <__main__.TestRotationMethod testMethod=test_hilbert>> 
 [[1.33226763e-15]
 [6.66133815e-16]
 [5.55111512e-16]
 [4.99600361e-16]
 [2.22044605e-16]]
TestRotationMethod
<bound method TestRotationMethod.test_square of <__main__.TestRotationMethod testMethod=test_square>> 
 [[ 0.00000000e+00]
 [ 0.00000000e+00]
 [-3.55271368e-

In [69]:
n = 5
a = np.array([
    [2, 3, -4, 1],
    [1, -2, -5, 1],
    [5, -3, 1, -4],
    [10, 2, -1, 2]
], dtype='float64')

b = np.array([[3, 2, 1, 4]], dtype='float64').T
x_calc = reflection_method(a, b)

In [70]:
print(x_calc)

[[ 0.33155823]
 [ 0.23714997]
 [-0.45411275]
 [-0.13698309]]


In [72]:
a_1 = np.array([[1/(i + j - 1) for j in range(1, n+1)] for i in range(1, n + 1)], dtype='float64')
x_1 = np.array([[-5, 6, 8, -2, -4]], dtype='float64').T
b_1 = np.matmul(a_1, x_1)
x_calc_1 = reflection_method(a_1, b_1)
print(x_calc_1)

[[-2.79258168]
 [ 0.76184989]
 [ 0.67504234]
 [ 0.56206945]
 [ 7.18503136]]
